In [104]:
from sklearn import tree
from sklearn.externals.six import StringIO
import quandl as ql
import pandas as pd
import math as math
import pydotplus

ql.ApiConfig.api_key = 'CbzNMfBFSFEFsiNNAe97'

In [105]:
#input NIKKEI and FXRate of USD/JPY
df_NIKKEI = ql.get('NIKKEI/INDEX')
df_FXRate = ql.get('FED/RXI_N_B_JA')

In [194]:
#make training data
#Nikkei data have newer data than that of FXRate
N=500
#M<N
M=470
#make classifier using M data, and test classifier using N-M data
df_concat = pd.concat([df_NIKKEI, df_FXRate], axis=1)
X = df_NIKKEI.tail(N+2)
Y = df_FXRate.tail(N)
Z = df_concat.tail(N)
samples = []
values = []
for i in range(1, M):
    #by using sum, I check where NaN exists in Z.ix[i,0:5]
    if math.isnan(sum(Z.ix[i,0:5])) == False and math.isnan(sum(Z.ix[i-1,0:5])) == False:
        #Today's Nikkei prices
        samples.append(list((Z.ix[i,0:4] - Z.ix[i-1,0:4]) / Z.ix[i-1, 0:4]))
        #Difference to yesterday's FXRate
        values = values + [int(Z.ix[i,4] - Z.ix[i-1,4])]

In [195]:
#make decision tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(samples, values)

In [196]:
#make test data
tests = []
answers = []
for i in range(M, N):
    if math.isnan(sum(Z.ix[i,0:5])) == False and math.isnan(sum(Z.ix[i-1,0:5])) == False:
        tests.append(list((Z.ix[i,0:4] - Z.ix[i-1,0:4]) / Z.ix[i-1, 0:4]))
        answers = answers + [int(Z.ix[i,4] - Z.ix[i-1,4])]

In [197]:
#result using decision tree
print(clf.predict(tests))

[ 0  0 -2 -1 -1  0  0  0  0  0  0 -1  0  0  0  0  0  0 -1  0  0 -2]


In [198]:
#Answer
print(answers)

[0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 1, -3]


In [199]:
dot_data=StringIO()
tree.export_graphviz(clf, out_file=dot_data)
graph=pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_pdf(r"FXRate.pdf")

True